# 6-DOF Airplane Simulation in Stability Coordinates

Model adapted from Stevens: Aircraft Control and Simulation

When building a nonlinear model of an aircraft, you can find the stability characteristics by

1. Finding a trim position of the aircraft (a set of thrust & control surface deflections that leads to equilibrium when the aircraft is in a given configuration)
2. Linearizing the equations of motion about the trim position
3. Decoupling the longitudinal and lateral / directional stability components if they are decoupled
4. Finding the poles (eigenvalues) of the A matrix to determine the stability of the aircraft
5. Determine which poles correspond to which dynamic modes

Then you can plot the behaviour of the aircraft around the given equilibrium point by forcing it with a 'doublet'.

We'll build a medium-fidelity nonlinear model of the aircraft by 
* discretizing the aircraft into particular aerodynamic surfaces (each wing, maybe split into multiple sections for which sections have airfoils and which ones do not)
* Calculating each surface individually
* Add explicit terms for interaction between surfaces, eg the propwash over the main wing and tail and the downwash from the main wing over the tail of the craft

The unfortunate fact about the nonlinear model of the craft is that it relies on a lot of aerodynamic data which you would really
want to measure in a wind tunnel. We'll use potential-flow modelling to get these aerodynamic coefficients as much as possible,
but we'll need to verify this performance against X-Plane and the physical model.

In [4]:
import numpy as np

def adc(vt, h):
    "Calculate mach number and dynamic pressure at speed v and altitude h"
    return vt / 316, 1/2 * 1.2 * vt**2

def transp(time, x, u):
    # medium sized transport aircraft, longitudinal dynamics
    S = 1270.0 # wing area, ft^2
    CBAR = 17.5 # chord, ft
    MASS = 5.0E3 # mass, slug
    IYY = 4.1E6 # mass moment of inertia about pitch axis
    TSTAT = 6.0E4 # idk
    DTDV = -38.0 # thrust variation with velocity (prop craft)
    ZE = 2.0 # thrust axis offset from center of mass
    CDCLS = 0.042 # drag polar
    CLA = 0.085 # wing lift curve slope per degree
    CMA = -0.022 # moment coefficient per degree
    CMDE = -0.016 # tail effectiveness per degree
    CMQ = -16.0 # aerodynamic ? per radian 
    CMADOT = -6.0 # aerodynamic ? per radian
    CLADOT = 0.0 # per radian
    GD = 32.17 # acceleration due to gravity, ft/s^2
    THTL, ELEV, XCG, LAND = u
    VT = x[0] # tas, 
    ALPHA = np.degrees(x[1]) # a.o.a, 
    THETA = x[2] # pitch attitude, 
    Q = x[3] # pitch rate, 
    H = x[4] # altitude

    MACH, QBAR = adc(VT, H)
    QS = QBAR * S
    SALP = np.sin(x[1]) # ALP = ALPHA
    CALP = np.cos(x[1])
    GAM = THETA - x[2]; SGAM = np.sin(GAM); CGAM = np.cos(GAM)
    if LAND == 0:
        CL0 = 0.20; CD0 = 0.016; CM0 = 0.05; DCDG = 0.00; DCMG = 0; # clean
    elif LAND == 1:
        CL0 = 1.00; CD0 = 0.08; CM0 = -0.20; DCDG = 0.02; DCMG = -0.05 # landing flaps & gear
    else:
        raise Exception("Landing gear & flaps?")
    THR = (TSTAT + VT * DTDV) * np.maximum(THTL, 0) # thrust
    CL = CL0 + CLA * ALPHA # nondim. lift
    CM = DCMG + CM0 + CMA * ALPHA + CMDE * ELEV + CL * (XCG - 0.25) # moment
    CD = DCDG + CD0 + CDCLS * CL**2 # drag polar

    
    # state equations next
    xd = np.zeros(6)
    xd[0] = (THR * CALP - QS * CD) / MASS - GD * SGAM # forward velocity
    xd[1] = (-THR * SALP - QS * CL + MASS * (VT * Q + GD * CGAM)) / (MASS * VT + QS * CLADOT)
    xd[2] = Q
    D = 0.5 * CBAR * (CMQ * Q + CMADOT * xd[1]) / VT # pitch damping
    xd[3] = (QS * CBAR * (CM + D) + THR * ZE) / IYY # Qdot
    xd[4] = VT * SGAM # vertical speed
    xd[5] = VT * CGAM # horizontal speed
    return xd
    
    